# The Aiyagari model

Here we follow the work of Aiyagari and Huggett. There are many resources to follow here. We will look at a few different implementations in Julia. We have done value function iteration before, this is the simplest method, so we will follow it here as well initially. 

### Model description

Aggregate behaviour is result of market interaction among large number of agents subject to idiosyncratic shocks. This involves asset trading as the main mechanism by which agents smooth consumption. There is also individual dynamics and uncertainty in this model. 

Aggregate variables are unchanging in this model. 

Model is built around the standard neoclassical growth model and is modified to include a role for uninsured idiosyncratic risk and liquidity / borrowing constraints.

### Model ingredients

Typical consumption savings problem in infinite horizon. There are two important features of this model. 

1. Idiosyncratic shocks -- The individual receives exogenous income shocks.
2. Incomplete markets -- The individuals cannot trade assets, there is no insurance in this market

We first deal with the case where there is no aggregate uncertainty.

This means that individuals are ex-ante the same, but ex-post they might differ.

Importantly, the input into this process, the earnings process, is exogenous. However, the wealth distribution generated from this process is endogenous.. 